阶段1：构造模型

在票价确定的前提下，预测各航段的客流量（已完成）。

阶段2：解析过程

根据客流信息分配舱位，以确定总收益。
客舱分为两类：短途类（AB或BC，称为S类）和长途类（AC，称为L类），总容量为C。

根据是否满座进行分类：

若不满座（S+L≤C），则按S、L和C的预测比例分配舱位，以实现收益最大化。
若满座（S+L>C），则需根据销售量变化预测进行取舍，并作如下判断：
若票价AB+BC>AC，则优先将舱位拆分为AB和BC段销售，S的值取min(AB, BC)，其余舱位分配给L。
若票价AB+BC<AC，则优先分配舱位给AC段，剩余舱位分配给AB和BC段。

阶段3：求总收益最大值

以票价为自变量，总收益为因变量，绘制总收益曲线，并求出总收益的最大值。

In [1]:
! which python

/data/zy/anaconda3/envs/machine/bin/python


# 加载CSV数据

## 加载CSV文件

In [2]:
import pandas as pd
import numpy as np

# 读取CSV文件
# /Users/zhanyu/code/data-hh/海航系销售结果数据
# df = pd.read_csv('../train_data/海航系销售结果数据.csv')
df = pd.read_csv('data_hh/result/cap_补全后.csv',index_col=0)
# df = pd.read_csv('data_hh/航班销售结果数据_2023-01-01_2025-07-01_encrypt.csv',index_col=0)

# 显示前两行数据以确保正确加载
print(df.shape)
print(df.head(5))
print(df.tail(5))

(2096151, 14)
     flt_date        segment  flt_no  dep_time    cap aircraft  legs  leg_no  \
0  2023-01-01  a9f275-24cc4b  3c6705  14:35:00  110.0      195     1       1   
1  2023-01-01  32edcc-d56796  5cb692  14:05:00   94.0      190     3       2   
2  2023-01-01  d56796-ec1b13  de569b  18:00:00   94.0      190     3       1   
3  2023-01-01  854d0a-d56796  de5522  18:50:00   94.0      190     3       2   
4  2023-01-01  d56796-24cc4b  6a67d9  22:40:00  110.0      195     1       1   

   duration  tkt_rev  pax       a       b       c  
0      1.30  45636.0   97  a9f275  24cc4b  7f9c2b  
1      1.23   3700.0    7  32edcc  d56796  854d0a  
2      1.50   1510.0    2  d56796  ec1b13  dcf90a  
3      0.70    620.0    2  854d0a  d56796  32edcc  
4      1.25  30480.0   67  d56796  24cc4b  7f9c2b  
           flt_date        segment  flt_no  dep_time    cap aircraft  legs  \
2096146  2025-07-01  e5aeda-08f792  2f5ac7  13:10:00  161.0      738     3   
2096147  2025-07-01  08f792-1231a3  8

现在的数据相对于之前的缺失了bd_type字段，理论上来说补充更好，但是这个另一个角度来说，有aircraft就够了，我们暂时先不补充这一点，有时间了再补充

## 查看一些字段的统计情况

### 对flt_no字段进行了查看

In [3]:
# 获取第2列的列名
first_column_name = df.columns[2]
print("第0列的列名:", first_column_name)

# 查看第2列的唯一值类型
unique_types = df.iloc[:, 2].apply(type).unique()
print("\n第0列中的数据类型：")
print(unique_types)

# 查看第2列的唯一值
unique_values = df.iloc[:, 2].unique()
print("\n第0列中的唯一值示例：")
print(unique_values[:-1])  # 显示前10个唯一值

# 统计第2列每个取值的数量
value_counts = df.iloc[:, 2].value_counts()

# 输出每种取值及其数量
print("\n第2列的取值及其数量：")
print(value_counts)

第0列的列名: flt_no

第0列中的数据类型：
[<class 'str'>]

第0列中的唯一值示例：
['3c6705' '5cb692' 'de569b' ... '4e0f13' '904919' 'f29522']

第2列的取值及其数量：
flt_no
e314e5    4170
f3ca4a    4168
f53fa1    3521
ff9f48    3501
26ed31    3427
          ... 
1b6336       1
dbd072       1
df3a7c       1
824fb5       1
397c06       1
Name: count, Length: 4894, dtype: int64


### 对legs字段进行了统计

In [4]:
# 获取 'legs' 列的列名
legs_column_name = 'legs'  # 或者可以使用 df.columns[2] 如果 'legs' 是第3列
print(f"列名: {legs_column_name}")

# 查看 'legs' 列的唯一值类型
unique_types = df[legs_column_name].apply(type).unique()
print("\n'legs' 列中的数据类型：")
print(unique_types)

# 查看 'legs' 列的唯一值
unique_values = df[legs_column_name].unique()
print("\n'legs' 列中的唯一值示例：")
print(unique_values[:10])  # 显示前10个唯一值（你可以调整数量）

# 统计 'legs' 列每个取值的数量
value_counts = df[legs_column_name].value_counts()

# 输出每种取值及其数量
print("\n'legs' 列的取值及其数量：")
print(value_counts)

列名: legs

'legs' 列中的数据类型：
[<class 'int'>]

'legs' 列中的唯一值示例：
[1 3 4]

'legs' 列的取值及其数量：
legs
1    1152295
3     943605
4        251
Name: count, dtype: int64


### 对leg_no字段进行了统计

In [5]:
# 获取 'leg_no' 列的列名
leg_no_column_name = 'leg_no'  # 如果列名不是 'leg_no'，请更改为实际的列名
print(f"列名: {leg_no_column_name}")

# 查看 'leg_no' 列的唯一值类型
unique_types = df[leg_no_column_name].apply(type).unique()
print("\n'leg_no' 列中的数据类型：")
print(unique_types)

# 查看 'leg_no' 列的唯一值
unique_values = df[leg_no_column_name].unique()
print("\n'leg_no' 列中的唯一值示例：")
print(unique_values[:10])  # 显示前10个唯一值（你可以调整数量）

# 统计 'leg_no' 列每个取值的数量
value_counts = df[leg_no_column_name].value_counts()

# 输出每种取值及其数量
print("\n'leg_no' 列的取值及其数量：")
print(value_counts)

列名: leg_no

'leg_no' 列中的数据类型：
[<class 'int'>]

'leg_no' 列中的唯一值示例：
[1 2 3 4]

'leg_no' 列的取值及其数量：
leg_no
1    1466869
3     314604
2     314552
4        126
Name: count, dtype: int64


### 对duration字段进行了统计

In [6]:
# 获取 'duration' 列的列名
duration_column_name = 'duration'  # 如果列名不是 'duration'，请更改为实际的列名
print(f"列名: {duration_column_name}")

# 查看 'duration' 列的唯一值类型
unique_types = df[duration_column_name].apply(type).unique()
print("\n'duration' 列中的数据类型：")
print(unique_types)

# 查看 'duration' 列的唯一值
unique_values = df[duration_column_name].unique()
print("\n'duration' 列中的唯一值示例：")
print(unique_values[:10])  # 显示前10个唯一值（你可以调整数量）

# 统计 'duration' 列每个取值的数量
value_counts = df[duration_column_name].value_counts()

# 输出每种取值及其数量
print("\n'duration' 列的取值及其数量：")
print(value_counts)

列名: duration

'duration' 列中的数据类型：
[<class 'float'>]

'duration' 列中的唯一值示例：
[1.3  1.23 1.5  0.7  1.25 4.75 1.83 3.98 1.45 3.02]

'duration' 列的取值及其数量：
duration
2.25     20244
2.08     19913
2.17     19865
2.20     19491
2.18     19394
         ...  
30.52        1
1.19         1
0.38         1
10.33        1
9.47         1
Name: count, Length: 919, dtype: int64


# 依次处理特征

## 拆分 flt_date 为 year, month, day, weekday

In [7]:
# 将 'flt_date' 列转换为 datetime 格式
df['flt_date'] = pd.to_datetime(df['flt_date'])

# # 提取年、月、日和星期几
# df['year'] = df['flt_date'].dt.year
# df['month'] = df['flt_date'].dt.month
# df['day'] = df['flt_date'].dt.day
# df['weekday'] = df['flt_date'].dt.weekday  # 0 = Monday, 6 = Sunday

# # 删除原始的 'flt_date' 字段
# df = df.drop(columns=['flt_date'])

# print(df.shape)
# print(df.head(5))
# print(df.tail(5))

## 拆分 dep_time 为 hour, minute, second

我们统计后发现所有行的second字段全是0，所以这个特征没用，所以删了

In [8]:
# # 拆分 'dep_time' 字段
# df[['hour', 'minute', 'second']] = df['dep_time'].str.split(':', expand=True)

# # 将拆分的结果转换为整数
# df['hour'] = df['hour'].astype(int)
# df['minute'] = df['minute'].astype(int)
# df['second'] = df['second'].astype(int)

# # 删除原始的 'dep_time' 字段
# df = df.drop(columns=['dep_time'])

# # 删除 'second' 字段
# df = df.drop(columns=['second'])

# # 查看结果
# print(df.shape)
# print(df.head(5))
# print(df.tail(5))

## 拆分 segment 为 from 和 to

对segment进行了统计，所有的segment长度都为6，没有不对的segment

In [9]:
# # 提取 'segment' 列的前三个字符作为 'from' 列
# df['from'] = df['segment'].str[:3]

# # 提取 'segment' 列的后三个字符作为 'to' 列
# df['to'] = df['segment'].str[-3:]

# # 删除原始的 'segment' 字段
# df = df.drop(columns=['segment'])

# # 查看结果
# print(df.shape)
# print(df.head(2))
# print(df.tail(2))

## 删除pax异常的行

In [10]:
# 统计 'pax' 字段中缺失值的行数
missing_pax = df[df['pax'].isna()]

# 统计 'pax' 字段中0值的行数
zero_pax = df[df['pax'] == 0]

# 输出统计结果
num_missing = missing_pax.shape[0]
num_zero = zero_pax.shape[0]

print(df.shape[0])
print(f"缺失值的行数: {num_missing}")
print(f"为0的行数: {num_zero}")

2096151
缺失值的行数: 0
为0的行数: 13022


In [11]:
# 删除 'pax' 字段为缺失值或为0的行
df = df.dropna(subset=['pax'])  # 删除pax列中的缺失值行
df = df[df['pax'] != 0]  # 删除pax列中为0的行

# 查看删除后的DataFrame行数
num_rows_after_cleanup = df.shape[0]
print(f"删除缺失值或为0的行后，DataFrame一共有 {num_rows_after_cleanup} 行")

删除缺失值或为0的行后，DataFrame一共有 2083129 行


In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def remove_outliers(data, col):
    """删除指定列中的异常值"""
    Q1 = data[col].quantile(0.25)  # 第1四分位数
    Q3 = data[col].quantile(0.75)  # 第3四分位数
    IQR = Q3 - Q1  # 四分位距
    lower_bound = Q1 - 1.5 * IQR  # 下限
    upper_bound = Q3 + 1.5 * IQR  # 上限
    
    # 筛选出范围内的数据
    filtered_data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]
    return filtered_data

def plot(data, col):
    """绘制指定列的分布图和箱型图"""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
    sns.histplot(data[col], kde=True, ax=ax1)  # 替换 distplot 为 histplot
    sns.boxplot(data[col], ax=ax2)
    plt.tight_layout()
    plt.show()

# # 绘制过滤后的数据分布图和箱型图
# plot(df, 'pax')
# # 示例数据框
# # 假设 df 是你的数据框
# # 删除 pax 列中的异常值
# # df = remove_outliers(df, 'pax')

# # 绘制过滤后的数据分布图和箱型图
# plot(df, 'pax')


In [13]:
# 删除 pax 小于 20 的行
df = df[df['pax'] >= 20]

# 查看结果
print(df.shape)
print(df.head(2))
print(df.tail(2))

(1972571, 14)
    flt_date        segment  flt_no  dep_time    cap aircraft  legs  leg_no  \
0 2023-01-01  a9f275-24cc4b  3c6705  14:35:00  110.0      195     1       1   
4 2023-01-01  d56796-24cc4b  6a67d9  22:40:00  110.0      195     1       1   

   duration  tkt_rev  pax       a       b       c  
0      1.30  45636.0   97  a9f275  24cc4b  7f9c2b  
4      1.25  30480.0   67  d56796  24cc4b  7f9c2b  
          flt_date        segment  flt_no  dep_time    cap aircraft  legs  \
2096149 2025-07-01  08f792-dcf90a  0bd968  11:45:00  188.0      7MZ     1   
2096150 2025-07-01  08f792-dcf90a  2befc2  20:55:00  161.0      738     1   

         leg_no  duration   tkt_rev  pax       a       b       c  
2096149       1      2.40  107720.0  144  08f792  dcf90a  7f9c2b  
2096150       1      2.48  105060.0  142  08f792  dcf90a  7f9c2b  


## 处理aircraft

aircraft全是str类型，且没有数据缺失的情况

In [14]:
# 查看 'cap' 列中每个元素的数据类型
print(df['aircraft'].apply(type).value_counts())

aircraft
<class 'str'>    1972571
Name: count, dtype: int64


In [15]:
# 统计 'aircraft' 字段中缺失值的行数
missing_pax = df[df['aircraft'].isna()]

# 统计 'aircraft' 字段中0值的行数
zero_pax = df[df['aircraft'] == 0]

# 输出统计结果
num_missing = missing_pax.shape[0]
num_zero = zero_pax.shape[0]

print(df.shape[0])
print(f"缺失值的行数: {num_missing}")
print(f"为0的行数: {num_zero}")

1972571
缺失值的行数: 0
为0的行数: 0


In [16]:
# 统计 'aircraft' 字段的不同取值及其数量
aircraft_counts = df['aircraft'].value_counts()

# 输出统计结果
print("aircraft 字段的不同取值及其数量：")
print(aircraft_counts)

aircraft 字段的不同取值及其数量：
aircraft
738    573115
320    283773
190    130325
195     75905
73V     64434
        ...  
33A       359
33N       246
33K       236
7C1       226
3C1        14
Name: count, Length: 72, dtype: int64


## 处理legs和leg_np异常行

In [17]:
# 删除 'legs' 列中值为4的行
df = df[df['legs'] != 4]

# 查看结果
print(df.shape)
print(df.head(2))
print(df.tail(2))

(1972416, 14)
    flt_date        segment  flt_no  dep_time    cap aircraft  legs  leg_no  \
0 2023-01-01  a9f275-24cc4b  3c6705  14:35:00  110.0      195     1       1   
4 2023-01-01  d56796-24cc4b  6a67d9  22:40:00  110.0      195     1       1   

   duration  tkt_rev  pax       a       b       c  
0      1.30  45636.0   97  a9f275  24cc4b  7f9c2b  
4      1.25  30480.0   67  d56796  24cc4b  7f9c2b  
          flt_date        segment  flt_no  dep_time    cap aircraft  legs  \
2096149 2025-07-01  08f792-dcf90a  0bd968  11:45:00  188.0      7MZ     1   
2096150 2025-07-01  08f792-dcf90a  2befc2  20:55:00  161.0      738     1   

         leg_no  duration   tkt_rev  pax       a       b       c  
2096149       1      2.40  107720.0  144  08f792  dcf90a  7f9c2b  
2096150       1      2.48  105060.0  142  08f792  dcf90a  7f9c2b  


In [18]:
# 获取 'legs' 列的列名
legs_column_name = 'legs'  # 或者可以使用 df.columns[2] 如果 'legs' 是第3列
print(f"列名: {legs_column_name}")

# 查看 'legs' 列的唯一值类型
unique_types = df[legs_column_name].apply(type).unique()
print("\n'legs' 列中的数据类型：")
print(unique_types)

# 查看 'legs' 列的唯一值
unique_values = df[legs_column_name].unique()
print("\n'legs' 列中的唯一值示例：")
print(unique_values[:10])  # 显示前10个唯一值（你可以调整数量）

# 统计 'legs' 列每个取值的数量
value_counts = df[legs_column_name].value_counts()

# 输出每种取值及其数量
print("\n'legs' 列的取值及其数量：")
print(value_counts)

列名: legs

'legs' 列中的数据类型：
[<class 'int'>]

'legs' 列中的唯一值示例：
[1 3]

'legs' 列的取值及其数量：
legs
1    1149661
3     822755
Name: count, dtype: int64


In [19]:
# 获取 'leg_no' 列的列名
leg_no_column_name = 'leg_no'  # 如果列名不是 'leg_no'，请更改为实际的列名
print(f"列名: {leg_no_column_name}")

# 查看 'leg_no' 列的唯一值类型
unique_types = df[leg_no_column_name].apply(type).unique()
print("\n'leg_no' 列中的数据类型：")
print(unique_types)

# 查看 'leg_no' 列的唯一值
unique_values = df[leg_no_column_name].unique()
print("\n'leg_no' 列中的唯一值示例：")
print(unique_values[:10])  # 显示前10个唯一值（你可以调整数量）

# 统计 'leg_no' 列每个取值的数量
value_counts = df[leg_no_column_name].value_counts()

# 输出每种取值及其数量
print("\n'leg_no' 列的取值及其数量：")
print(value_counts)

列名: leg_no

'leg_no' 列中的数据类型：
[<class 'int'>]

'leg_no' 列中的唯一值示例：
[1 3 2]

'leg_no' 列的取值及其数量：
leg_no
1    1453887
2     303541
3     214988
Name: count, dtype: int64


## 处理cap(待做)

查看了数据后发现cap全是int类型，没有异常

cap=0的情况大部分都是leg_no=3的时候，可以根据leg_no=1和2或者aircraft补全数据

In [20]:
# 查看 'cap' 列中每个元素的数据类型
print(df['cap'].apply(type).value_counts())

cap
<class 'float'>    1972416
Name: count, dtype: int64


In [21]:
# 统计 'pax' 字段中缺失值的行数
missing_pax = df[df['cap'].isna()]

# 统计 'pax' 字段中0值的行数
zero_pax = df[df['cap'] == 0]

# 输出统计结果
num_missing = missing_pax.shape[0]
num_zero = zero_pax.shape[0]

print(df.shape[0])
print(f"缺失值的行数: {num_missing}")
print(f"为0的行数: {num_zero}")

# # 将 'cap' 字段中为0的值修改为 NaN
# df.loc[df['cap'] == 0, 'cap'] = np.nan

# # 将 'cap' 字段转换为字符串类型
# df['cap'] = df['cap'].astype(str)



1972416
缺失值的行数: 0
为0的行数: 9


In [22]:
# 统计 'cap' 字段中为0的行
zero_cap = df[df['cap'] == 0]

# 统计这些 'cap' 为0的行中，'leg_no' 字段的不同取值及其数量
leg_no_counts = zero_cap['leg_no'].value_counts()

# 输出统计结果
print(f"总行数: {df.shape[0]}")
print(f"cap 为0的行数: {zero_cap.shape[0]}")
print("在 'cap' 为0的行中，'leg_no' 字段的不同取值及其数量：")
print(leg_no_counts)

总行数: 1972416
cap 为0的行数: 9
在 'cap' 为0的行中，'leg_no' 字段的不同取值及其数量：
leg_no
3    4
1    4
2    1
Name: count, dtype: int64


In [23]:
# 删除 cap 为 0 的所有行
df = df[df['cap'] != 0]

# 统计 'cap' 字段中为0的行
zero_cap = df[df['cap'] == 0]

# 统计这些 'cap' 为0的行中，'leg_no' 字段的不同取值及其数量
leg_no_counts = zero_cap['leg_no'].value_counts()

# 输出统计结果
print(f"总行数: {df.shape[0]}")
print(f"cap 为0的行数: {zero_cap.shape[0]}")
print("在 'cap' 为0的行中，'leg_no' 字段的不同取值及其数量：")
print(leg_no_counts)

总行数: 1972407
cap 为0的行数: 0
在 'cap' 为0的行中，'leg_no' 字段的不同取值及其数量：
Series([], Name: count, dtype: int64)


## 处理duration（待做）

数据有少量缺失，感觉可以通过segment的历史信息补全这部分信息

In [24]:
# 统计 'duration' 字段中缺失值的行数
missing_duration = df[df['duration'].isna()]

# 统计 'duration' 字段中为0值的行数
zero_duration = df[df['duration'] == 0]

# 输出统计结果
num_missing_duration = missing_duration.shape[0]
num_zero_duration = zero_duration.shape[0]

# 总行数
print(f"总行数: {df.shape[0]}")
print(f"duration 字段缺失值的行数: {num_missing_duration}")
print(f"duration 字段为0的行数: {num_zero_duration}")

总行数: 1972407
duration 字段缺失值的行数: 0
duration 字段为0的行数: 406


In [25]:
# 过滤出 'duration' 为 0 的行
zero_duration = df[df['duration'] == 0]

# 统计这些 'duration' 为 0 的行中 'leg_no' 字段的不同取值及其数量
leg_no_counts = zero_duration['leg_no'].value_counts()

# 输出统计结果
print(f"duration 为 0 的行中，'leg_no' 字段的不同取值及其数量：")
print(leg_no_counts)

duration 为 0 的行中，'leg_no' 字段的不同取值及其数量：
leg_no
1    271
3     71
2     64
Name: count, dtype: int64


In [26]:
# 将 'duration' 列中值为 0 的部分替换为 NaN
df.loc[df['duration'] == 0, 'duration'] = np.nan

## 处理tkt_rev

有一些异常行的tkt_rev为0，pax却不为0

In [27]:
# 统计 'unit_price' 列中为 0 的行数
zero_count = (df['tkt_rev'] == 0).sum()

# 统计 'unit_price' 列中为 NaN 的行数
nan_count = df['tkt_rev'].isna().sum()

# 输出结果
print(f"tkt_rev 列中为 0 的行数: {zero_count}")
print(f"tkt_rev 列中为 NaN 的行数: {nan_count}")

tkt_rev 列中为 0 的行数: 157
tkt_rev 列中为 NaN 的行数: 0


In [28]:
# 过滤出 'tkt_rev' 为 0 的行
zero_tkt_rev = df[df['tkt_rev'] == 0]

# 统计这些 'tkt_rev' 为 0 的行中 'leg_no' 字段的不同取值及其数量
leg_no_counts = zero_tkt_rev['leg_no'].value_counts()

# 输出统计结果
print(f"tkt_rev 为 0 的行中，'leg_no' 字段的不同取值及其数量：")
print(leg_no_counts)

tkt_rev 为 0 的行中，'leg_no' 字段的不同取值及其数量：
leg_no
1    111
2     41
3      5
Name: count, dtype: int64


In [29]:
df = df[df['tkt_rev'] != 0]

# 计算单价 'unit_price'，即 tkt_rev 除以 pax
df['unit_price'] = df['tkt_rev'] / df['pax']

# 删除 'tkt_rev' 列
df = df.drop(columns=['tkt_rev'])

# 查看结果
print(df.shape)
print(df.head(2))
print(df.tail(2))

(1972250, 14)
    flt_date        segment  flt_no  dep_time    cap aircraft  legs  leg_no  \
0 2023-01-01  a9f275-24cc4b  3c6705  14:35:00  110.0      195     1       1   
4 2023-01-01  d56796-24cc4b  6a67d9  22:40:00  110.0      195     1       1   

   duration  pax       a       b       c  unit_price  
0      1.30   97  a9f275  24cc4b  7f9c2b  470.474227  
4      1.25   67  d56796  24cc4b  7f9c2b  454.925373  
          flt_date        segment  flt_no  dep_time    cap aircraft  legs  \
2096149 2025-07-01  08f792-dcf90a  0bd968  11:45:00  188.0      7MZ     1   
2096150 2025-07-01  08f792-dcf90a  2befc2  20:55:00  161.0      738     1   

         leg_no  duration  pax       a       b       c  unit_price  
2096149       1      2.40  144  08f792  dcf90a  7f9c2b  748.055556  
2096150       1      2.48  142  08f792  dcf90a  7f9c2b  739.859155  


## 保存文件

In [30]:
# 查看前几行数据，确保加载成功
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1972250 entries, 0 to 2096150
Data columns (total 14 columns):
 #   Column      Dtype         
---  ------      -----         
 0   flt_date    datetime64[ns]
 1   segment     object        
 2   flt_no      object        
 3   dep_time    object        
 4   cap         float64       
 5   aircraft    object        
 6   legs        int64         
 7   leg_no      int64         
 8   duration    float64       
 9   pax         int64         
 10  a           object        
 11  b           object        
 12  c           object        
 13  unit_price  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(7)
memory usage: 225.7+ MB
None


In [31]:
df = df.reset_index(drop=True)

# 保存处理后的数据到新的CSV文件
df.to_csv('data_hh/result/pre_2023-2025.csv', index=False)

In [32]:


# 查看结果
print(df.shape)
print(df.head(2))
print(df.tail(2))

(1972250, 14)
    flt_date        segment  flt_no  dep_time    cap aircraft  legs  leg_no  \
0 2023-01-01  a9f275-24cc4b  3c6705  14:35:00  110.0      195     1       1   
1 2023-01-01  d56796-24cc4b  6a67d9  22:40:00  110.0      195     1       1   

   duration  pax       a       b       c  unit_price  
0      1.30   97  a9f275  24cc4b  7f9c2b  470.474227  
1      1.25   67  d56796  24cc4b  7f9c2b  454.925373  
          flt_date        segment  flt_no  dep_time    cap aircraft  legs  \
1972248 2025-07-01  08f792-dcf90a  0bd968  11:45:00  188.0      7MZ     1   
1972249 2025-07-01  08f792-dcf90a  2befc2  20:55:00  161.0      738     1   

         leg_no  duration  pax       a       b       c  unit_price  
1972248       1      2.40  144  08f792  dcf90a  7f9c2b  748.055556  
1972249       1      2.48  142  08f792  dcf90a  7f9c2b  739.859155  
